In [1]:
import pandas as pd
import os
os.environ['PYTHONHASHSEED']=str(2)

import numpy as np
from sklearn.utils import class_weight
from keras_preprocessing import image as im
import tensorflow as tf
import random

In [2]:
def reset_random_seeds():
   os.environ['PYTHONHASHSEED']=str(2)
   np.random.seed(2)
   random.seed(2)
   torch.manual_seed(2)


In [3]:
from sentence_transformers import SentenceTransformer, losses
from sentence_transformers import models
from torch import nn
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel


# Loading and preparing data

In [4]:
df = pd.read_csv('../../../data/emo-at-cap/emo-at-cap.csv')

In [5]:
df.columns

Index(['image_name', 'annotation', 'human_sentiment', 'sentiment'], dtype='object')

In [6]:
annotations = df['annotation'].str.lower().values
sentiment = df['human_sentiment'].values

In [7]:
sentiment

array(['Neutral', 'Positive', 'Positive', ..., 'Positive', 'Negative',
       'Negative'], dtype=object)

In [8]:
mapping_sen = {'Negative' : 0, 'Neutral' : 1, 'Positive' : 2}

# Model 

In [9]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/distilbert-base-nli-stsb-mean-tokens')


In [10]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [11]:
class BertSA(nn.Module):
    def __init__(self, finetune=False):
        super(BertSA, self).__init__()
        reset_random_seeds()
        self.bert = AutoModel.from_pretrained('sentence-transformers/distilbert-base-nli-stsb-mean-tokens')
        if not finetune:
            for param in self.bert.parameters():
                param.requires_grad = False

        ### New layers:
        self.linear1 = nn.Linear(768, 256)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(256, 3) ## as you have 3 classes in the output

    def forward(self, input_ids, attention_mask):
        encoded = self.bert(input_ids,attention_mask=attention_mask)
        encoded = mean_pooling(encoded, attention_mask)
        linear1_out = self.relu(self.linear1(encoded))
        linear2_out = self.linear2(linear1_out)
        return linear2_out

In [12]:
model = BertSA()


# Training 

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_val, y_train, y_val = train_test_split(annotations, sentiment , random_state=0, test_size=0.2)
X_val, X_test, y_val , y_test = train_test_split(X_val,y_val , random_state=0, test_size=0.5)

In [15]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, X,y, tokenizer):
        self.tokenizer = tokenizer
        self.mapping_sen =  {'Negative' : 0, 'Neutral' : 1, 'Positive' : 2}
        X = self.map_X(list(X))
        self.X, self.mask = X['input_ids'], X['attention_mask']
        self.y = self.map_y(y)
    
    def map_X(self, X):
        return self.tokenizer(X, padding=True, truncation=True, return_tensors='pt')
    
    def map_y(self, y):
        return torch.tensor([self.mapping_sen[i] for i in y])
    
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        return self.X[idx], self.mask[idx], self.y[idx]

In [16]:
train_dataset = SentimentDataset(X_train, y_train, tokenizer)
validation_dataset = SentimentDataset(X_val, y_val, tokenizer)

In [17]:
epochs = 25
batch_size = 128

In [18]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [19]:
device = torch.device("cpu")
model.to(device)

BertSA(
  (bert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_features=768

In [20]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score

In [21]:
overall_loss = []
for epoch in tqdm(range(epochs)):  # loop over the dataset multiple times
    tmp_train_loss = []
    train_predictions = []
    all_labels = []
    model.train()
    for train_idx in range(0, len(train_dataset), batch_size):
        # get the inputs; data is a list of [inputs, labels]
        inputs, mask, labels = train_dataset[train_idx:train_idx+batch_size]
        all_labels.extend(labels.cpu().detach().numpy())
        inputs, mask, labels = inputs.to(device), mask.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs, mask)
        train_predictions.extend(np.argmax(outputs.cpu().detach().numpy(),axis=1))
        loss_result = loss(outputs, labels)
        loss_result.backward()
        optimizer.step()

        # print statistics
        tmp_train_loss.append(loss_result.item())
        
    epoch_loss = np.mean(tmp_train_loss)
    train_accuracy = accuracy_score(y_true=all_labels, y_pred=train_predictions)
    
    print('Training loss : {}, Training accuracy : {}'.format(epoch_loss, train_accuracy))
    model.eval()
    with torch.no_grad():
         val_predictions = []
         tmp_val_loss = []
         all_labels = []

         for val_idx in range(0, len(validation_dataset), batch_size):
            inputs, mask, labels = validation_dataset[val_idx:val_idx+batch_size]
            all_labels.extend(labels.cpu().detach().numpy())

            inputs, mask, labels = inputs.to(device), mask.to(device), labels.to(device)
            outputs = model(inputs, mask)
            
            val_loss_result = loss(outputs, labels)
            tmp_val_loss.append(val_loss_result)
            
            val_predictions.extend(np.argmax(outputs.cpu().detach().numpy(), axis=1))
            
         epoch_val_loss = np.mean(tmp_train_loss)
         val_accuracy = accuracy_score(y_true=all_labels, y_pred=val_predictions)

         print('Val loss : {}, Val accuracy : {}'.format(epoch_val_loss, val_accuracy))

   

print('Finished Training')

  0%|          | 0/25 [00:00<?, ?it/s]

Training loss : 0.431018453091383, Training accuracy : 0.8486328125


  4%|▍         | 1/25 [00:41<16:47, 41.97s/it]

Val loss : 0.431018453091383, Val accuracy : 0.8828125
Training loss : 0.2864858365307252, Training accuracy : 0.9000651041666666


  8%|▊         | 2/25 [01:23<15:59, 41.71s/it]

...

Val loss : 0.02938339700146268, Val accuracy : 0.9036458333333334
Training loss : 0.028957261859128874, Training accuracy : 0.9928385416666666


 96%|█████████▌| 24/25 [16:43<00:42, 42.05s/it]

Val loss : 0.028957261859128874, Val accuracy : 0.8958333333333334
Training loss : 0.020991014627118904, Training accuracy : 0.9967447916666666


100%|██████████| 25/25 [17:23<00:00, 41.73s/it]

Val loss : 0.020991014627118904, Val accuracy : 0.90625
Finished Training


# Evaluation

In [22]:
from sklearn.metrics import f1_score

In [23]:
test_dataset = SentimentDataset(X_test, y_test, tokenizer)

In [54]:
model.eval()
test_predictions = []
tmp_test_loss = []
all_labels = []
with torch.no_grad():

     for test_idx in range(0, len(test_dataset), batch_size):
        inputs, mask, labels = test_dataset[test_idx:test_idx+batch_size]
        all_labels.extend(labels.cpu().detach().numpy())

        inputs, mask, labels = inputs.to(device), mask.to(device), labels.to(device)
        outputs = model(inputs, mask)

        test_loss_result = loss(outputs, labels)
        tmp_test_loss.append(test_loss_result)

        test_predictions.extend(np.argmax(outputs.cpu().detach().numpy(), axis=1))

     test_loss = np.mean(tmp_test_loss)
     test_accuracy = accuracy_score(y_true=all_labels, y_pred=test_predictions)

     print('Test loss : {}, Test accuracy : {}'.format(test_loss, test_accuracy))


Test loss : 0.38545188307762146, Test accuracy : 0.921875


# Validating results

In [29]:
mapping_sen

{'Negative': 0, 'Neutral': 1, 'Positive': 2}

In [30]:
inverse_sen_mapping = dict([(v,k) for k,v in mapping_sen.items()])

In [43]:
def predict(model, sen_mapping, sentence):
    sentence = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
    model.eval()
    with torch.no_grad():  
        sentence = sentence.to('cpu')
        label = sen_mapping[np.argmax(model(**sentence).cpu().detach().numpy())]
    return label

In [48]:
predict(model, inverse_sen_mapping, 'happy people')

'Positive'

# Saving results

In [49]:
df = pd.DataFrame()

In [61]:
df['name'] = ['sa_lstm_stbs']
df['f1_macro'] = [f1_score(y_true=test_dataset.y.numpy(), y_pred=test_predictions, average='macro')]
df['f1_weighted'] = [f1_score(y_true=test_dataset.y.numpy(), y_pred=test_predictions, average='weighted')]
df['acc'] = [accuracy_score(y_true=test_dataset.y.numpy(), y_pred=test_predictions)]
df.to_csv('sa_lstm_stbs_logs.csv', index=False)

In [62]:
df

,name,f1_macro,f1_weighted,acc
0,sa_lstm_stbs,0.850671,0.918409,0.921875
